# A / B Testing + Uplift

### Project Goal: Does introducting a promo code cause higher Conversions? and by introducing that to whom causes the conversions?
### Dataset Source: https://archive.ics.uci.edu/dataset/352/online+retail

In [36]:
import pandas as pd 
import numpy as np 

import matplotlib.pyplot as plt 
import seaborn as sns

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 100)

sns.set_style("whitegrid")

In [37]:
df = pd.read_excel(r"C:\Users\vinee\OneDrive\Desktop\projects\A-B Testing\data\Online Retail.xlsx")

In [38]:
df.shape

(541909, 8)

In [39]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [40]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [41]:
df.isna().sum().sort_values(ascending= False)

CustomerID     135080
Description      1454
StockCode           0
InvoiceNo           0
Quantity            0
InvoiceDate         0
UnitPrice           0
Country             0
dtype: int64

In [42]:
df.duplicated().sum()

np.int64(5268)

In [43]:
df["InvoiceDate"].min(), df["InvoiceDate"].max()

(Timestamp('2010-12-01 08:26:00'), Timestamp('2011-12-09 12:50:00'))

In [44]:
df["InvoiceDate"][1]


Timestamp('2010-12-01 08:26:00')

In [45]:
df["InvoiceDateOnly"] = df["InvoiceDate"].dt.strftime("%m-%d-%Y")
df["InvoiceDateOnly"][1]

'12-01-2010'

In [46]:
df["InvoiceDateOnly"].dtype

dtype('O')

In [47]:
df.select_dtypes(include="object").nunique().sort_values(ascending=False)

InvoiceNo          25900
Description         4223
StockCode           4070
InvoiceDateOnly      305
Country               38
dtype: int64

In [48]:
df["CustomerID"].nunique()

4372

In [49]:
# removing customerId nulls as they are no use for the test
df_clean = df.dropna(subset=["CustomerID"])

In [50]:
df_clean.isna().sum().sort_values(ascending=False)

InvoiceNo          0
StockCode          0
Description        0
Quantity           0
InvoiceDate        0
UnitPrice          0
CustomerID         0
Country            0
InvoiceDateOnly    0
dtype: int64

In [51]:
df.describe()

,Quantity,InvoiceDate,UnitPrice,CustomerID
count,541909.000000,541909,541909.000000,406829.000000
mean,9.552250,2011-07-04 13:34:57.156386048,4.611114,15287.690570
min,-80995.000000,2010-12-01 08:26:00,-11062.060000,12346.000000
25%,1.000000,2011-03-28 11:34:00,1.250000,13953.000000
50%,3.000000,2011-07-19 17:17:00,2.080000,15152.000000
75%,10.000000,2011-10-19 11:27:00,4.130000,16791.000000
max,80995.000000,2011-12-09 12:50:00,38970.000000,18287.000000
std,218.081158,NaN,96.759853,1713.600303


In [52]:
df_clean.dtypes

InvoiceNo                  object
StockCode                  object
Description                object
Quantity                    int64
InvoiceDate        datetime64[ns]
UnitPrice                 float64
CustomerID                float64
Country                    object
InvoiceDateOnly            object
dtype: object

In [55]:
df_clean["InvoiceDate"] = pd.to_datetime(df_clean["InvoiceDate"], errors="coerce")
df_clean["CustomerID"] = df_clean["CustomerID"].astype("Int64").astype("string")
df_clean["TotalPrice"] = df_clean["Quantity"] * df_clean["UnitPrice"]

C:\Users\vinee\AppData\Local\Temp\ipykernel_38184\2789383400.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean["InvoiceDate"] = pd.to_datetime(df_clean["InvoiceDate"], errors="coerce")
C:\Users\vinee\AppData\Local\Temp\ipykernel_38184\2789383400.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean["CustomerID"] = df_clean["CustomerID"].astype("Int64").astype("string")
C:\Users\vinee\AppData\Local\Temp\ipykernel_38184\2789383400.py:3: SettingWithCopyWarning: 
A value is trying to be set on

In [56]:
df_clean.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,InvoiceDateOnly,TotalPrice
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,12-01-2010,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,12-01-2010,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,12-01-2010,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,12-01-2010,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,12-01-2010,20.34


In [57]:
df_clean.dtypes

InvoiceNo                  object
StockCode                  object
Description                object
Quantity                    int64
InvoiceDate        datetime64[ns]
UnitPrice                 float64
CustomerID         string[python]
Country                    object
InvoiceDateOnly            object
TotalPrice                float64
dtype: object

In [66]:
df_clean["InvoiceNo"].astype(str).str.startswith("C").sum()

np.int64(8905)

In [64]:
(df_clean["Quantity"] < 0).sum()

np.int64(8905)

In [67]:
df_clean = df_clean[df_clean["Quantity"] > 0]

In [68]:
df_clean.to_csv(r"C:\Users\vinee\OneDrive\Desktop\projects\A-B Testing\data\cleaned_data.csv")